In [1]:
import pandas as pd

movie_day8_df = pd.read_csv('./day8.csv', encoding='utf-8')

In [2]:
%%time
new_df = 1
director_filmo = pd.DataFrame(columns=("code", "filmo","movie_code"))

for index,row in movie_day8_df.iterrows():
    peopleNm_list = [] ### 매번 비우면서
    
    if '|' in getattr(row,"director_name"):
        peopleNm_list = (getattr(row,"director_code").split('|'))
    else:
        peopleNm_list.append(getattr(row,"director_code"))
        
    filmoNames = getattr(row,"movie_name")
    filmocode = getattr(row,"movie_code")
    for director in peopleNm_list:
        new_df+=1 ## actor_filmo row count
        director_filmo.loc[new_df] = [director,filmoNames,filmocode]

Wall time: 20.8 s


In [1]:
day14_result = pd.read_csv('./14dir_result.csv', encoding='euc-kr')

NameError: name 'pd' is not defined

In [4]:
%%time
import aiohttp
import asyncio
import lxml
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import requests

async def fetch(link,director_code,filmo_name,movie_code,index):
    async with ClientSession() as session:
        async with session.get(link) as response:
            html = await response.read()
            
            soup = BeautifulSoup(html,"lxml")

            tbody = soup.findAll('em', class_='tl')

            result = []

            for page in soup.find_all('p',class_='pageList pmt2'):
                page_num = list(page.strings)
                page_result = [i for i in page_num if i != '\n']
                
            for etcParam in page_result:
            
                url2 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleDtl.do?"

                param = "code=%s&sType=filmo&etcParam=%s"  % (director_code,etcParam)

                link = url2+param

                average = await get(link,filmo_name,result)
                                
                director_average_df.loc[index] = [director_code,
                                                  filmo_name,
                                                  movie_code,
                                                  average
                                                 ]
async def get(link,filmo_name,result):
    async with ClientSession() as session:
        async with session.get(link) as response:
            
            html = await response.read()
            
            soup = BeautifulSoup(html,"lxml")

            tr_tag = []
            real_tag = []
            for count in soup.findAll('dl'):
                if any(filmo_name in s for s in list(count.strings)):
                    pass
                elif any('감독' in x for x in list(count.strings)):
    #                 else:
                    tr_tag = tr_tag + list(count.strings)

            for elem in range(0, len(tr_tag)) :
                if tr_tag[elem] == '[공식통계]':
                    real_tag.append(tr_tag[elem+3])

            real_tag = [w.replace('명','') for w in real_tag] ## 명 없애기
            real_tag = [w.replace(',','') for w in real_tag] ## , 없애기
    #             real_tag = [w.replace('','0') for w in real_tag] ## , 없애기
            real_tag = list(filter(None, real_tag))    ## 빈값 지우기
            real_tag = list(map(int,real_tag))

            result.extend(real_tag)

            return int(average(result))

def average(values):
    if len(values) == 0:
        return 0
    return sum(values, 0.0) / len(values)
        
if __name__ == "__main__":
    count= 1
    length = 300
    tasks = []
    last = 0
    director_average_df = pd.DataFrame(columns=['code','filmo','movie_code','average_audience'])
    
    loop_director = asyncio.get_event_loop()
    
    for index,row in director_filmo.iterrows():

        filmo_name = getattr(row,'filmo')
        filmo_name = filmo_name[1:]
        director_code = getattr(row,'code')
        movie_code = getattr(row,'movie_code')
        
        if any(day15_result['movie_name'] == filmo_name):
            director_average_df.loc[index] = [director_code,
                                              filmo_name,
                                              movie_code,
                                              day15_result['director_sum'][day15_result['movie_name'] == filmo_name].item()
                                             ]
        else:
            
            param = "code=%s&sType=filmo"  % (director_code)

            url1 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleDtl.do?"

            link = url1 + param

            task = asyncio.ensure_future(fetch(link,director_code,filmo_name,movie_code,index))
            tasks.append(task)

            if count == length:
                print(count)
                loop_director.run_until_complete(asyncio.wait(tasks))
                tasks = []
                length+=300

            if last == len(director_filmo):
                loop_director.run_until_complete(asyncio.wait(tasks))
                tasks = []
                loop_director.close()
                
            count+=1

300
600
900
Wall time: 4min 24s


In [8]:
director_sum = pd.DataFrame(columns=['movie_name','director_average'])

for index,row in movie_day8_df.iterrows():
    movie_code = getattr(row,'movie_code')
    movie_name = getattr(row,'movie_name')
    try:
        director_sum.loc[index] = [movie_name,
                                   max(director_average_df['average_audience'][director_average_df['movie_code'] == movie_code])
                                  ]
    except Exception as e:
        print(e)
        print(movie_code)
        director_sum.loc[index] = [movie_name,'0']

max() arg is an empty sequence
20137042
max() arg is an empty sequence
20122227
max() arg is an empty sequence
20136202
max() arg is an empty sequence
20136301
max() arg is an empty sequence
20122233
max() arg is an empty sequence
20137864
max() arg is an empty sequence
20137865
max() arg is an empty sequence
20138125
max() arg is an empty sequence
20138183
max() arg is an empty sequence
20124872
max() arg is an empty sequence
20124609
max() arg is an empty sequence
20137541
max() arg is an empty sequence
20137082
max() arg is an empty sequence
20137373
max() arg is an empty sequence
20136781
max() arg is an empty sequence
20137965
max() arg is an empty sequence
20121097
max() arg is an empty sequence
20135542
max() arg is an empty sequence
20137866
max() arg is an empty sequence
20134066
max() arg is an empty sequence
20138441
max() arg is an empty sequence
20138961
max() arg is an empty sequence
20138343
max() arg is an empty sequence
20138734
max() arg is an empty sequence
20137522


In [7]:
director_average_df = pd.read_csv('./8dire_more.csv',encoding='euc-kr')

In [12]:
del movie_day8_df['director_sum']

In [13]:
movie_day8_df['director_sum'] = director_average_df['average_audience']

In [15]:
movie_day8_df.to_csv('./day8.csv', encoding='euc-kr')